In [1]:
import sys
import os
from datetime import datetime
from google.oauth2 import service_account
from googleapiclient import discovery
from googleapiclient.http import MediaFileUpload
from datetime import datetime
import pandas as pd
import numpy as np
import time
import pyodbc
print(os.path.dirname(sys.executable))

d:\data_sale\venv\Scripts


In [2]:
server = '115.165.164.234'
driver = 'SQL Server'
db1 = 'PhaNam_eSales_PRO'
tcon = 'no'
uname = 'duyvq'
pword = '123VanQuangDuy'
cnxn = pyodbc.connect(driver='{ODBC Driver 17 for SQL Server}', 
                      host=server, database=db1, trusted_connection=tcon,
                      user=uname, password=pword)
cursor = cnxn.cursor()

In [3]:
datenow = datetime.now().strftime("%Y%m%d")
param_1 = f"'20161213'"
param_2 = f"'{datenow}'"

In [4]:
query = f"EXEC [pr_AR_TrackingDebtConfirm]  @Fromdate={param_1}, @Todate={param_2}"
start=time.time()
df1 = pd.read_sql(query, cnxn, parse_dates=['DateOfOrder'])
df1.to_csv('doanhthutienmat_data1.csv', index = False)
end=time.time()
duration=end-start
print(duration)
cnxn.close()

20.458582639694214


In [5]:
df1 = pd.read_csv('doanhthutienmat_data1.csv', dayfirst=True, parse_dates=['DateOfOrder'])

In [6]:
df1['OldData'] = np.where(df1['DateOfOrder'].values < np.datetime64(datetime(2021, 5, 1)), True, False)
# df1['OldData'] = df1['DateOfOrder'].apply(lambda x: True if x < datetime.strptime('01/05/2021', "%d/%m/%Y") else False)

In [7]:
# def mapping_amt_release(OldData,OpeiningOrderAmt,OrdAmtRelease):
#     if OldData == True:
#         return OpeiningOrderAmt
#     else:
#         return OrdAmtRelease
df1['OrdAmtRelease2'] = np.where(df1['OldData'].values, df1['OpeiningOrderAmt'].values, df1['OrdAmtRelease'].values)
df1['CountOrdRelease2'] = np.where(df1['OldData'].values, df1['CountOpeningOrder'].values, df1['CountOrdRelease'].values)
# df1['OrdAmtRelease2'] = df1.apply(lambda x: mapping_amt_release(x['OldData'], x['OpeiningOrderAmt'],x['OrdAmtRelease']),axis=1)
# df1['CountOrdRelease2'] = df1.apply(lambda x: mapping_amt_release(x['OldData'], x['CountOpeningOrder'],x['CountOrdRelease']),axis=1)
print(df1['OrdAmtRelease2'].sum())

186456950626.0


In [8]:
df1.columns

Index(['BranchID', 'SlsperID', 'Position', 'SlsName', 'PaymentsForm', 'Terms',
       'OrderDate', 'OrderNbr', 'DateOfOrder', 'CountOpeningOrder',
       'OpeiningOrderAmt', 'CountOrdRelease', 'OrdAmtRelease',
       'DeliveredOrder', 'DeliveredOrderAmt', 'CountReturnOrd', 'ReturnOrdAmt',
       'CountDebtConfirm', 'DebConfirmAmt', 'CountDebtConfirmRelease',
       'DebConfirmAmtRelease', 'OldData', 'OrdAmtRelease2',
       'CountOrdRelease2'],
      dtype='object')

In [9]:
import numpy as np # linear algebra
import pandas as pd

In [10]:
groupbylist = ['OrderNbr', 'BranchID', 'Position', 'SlsperID', 'DateOfOrder','SlsName', 'Terms', 'PaymentsForm']

In [11]:
aggregate_dict = {
'OrdAmtRelease2':np.sum, 'DeliveredOrderAmt':np.sum, 'ReturnOrdAmt': np.sum,
'DebConfirmAmt': np.sum, 'DebConfirmAmtRelease': np.sum, 'CountReturnOrd': np.sum,
'CountOrdRelease2': np.sum, 'DeliveredOrder':np.sum,
'CountDebtConfirm': np.sum, 'CountDebtConfirmRelease':np.sum
}

In [12]:
df2 = df1.groupby(groupbylist).aggregate(aggregate_dict)
df2.head()

,,,,,,,,OrdAmtRelease2,DeliveredOrderAmt,ReturnOrdAmt,DebConfirmAmt,DebConfirmAmtRelease,CountReturnOrd,CountOrdRelease2,DeliveredOrder,CountDebtConfirm,CountDebtConfirmRelease
OrderNbr,BranchID,Position,SlsperID,DateOfOrder,SlsName,Terms,PaymentsForm,,,,,,,,,,
DH012017-13345,MR0001,LOG,MR0456,2017-01-18,Lê Hồng Phúc,Debt,TM,19749975.0,0.0,0.0,0.0,0.0,0,1,0,0,0
DH012017-13346,MR0001,LOG,MR0456,2017-01-18,Lê Hồng Phúc,Debt,TM,24359987.0,0.0,0.0,0.0,0.0,0,1,0,0,0
DH012017-13360,MR0001,LOG,MR0456,2017-01-18,Lê Hồng Phúc,Debt,TM,17499997.0,0.0,0.0,0.0,0.0,0,1,0,0,0
DH012019-05622,MR0003,LOG,MR2168,2019-01-10,Nguyễn Anh Tuấn,Debt,TM,6150000.0,0.0,0.0,0.0,0.0,0,1,0,0,0
DH012020-00503,MR0016,MDS,MR1732,2020-01-02,Lâm Ngô Cường,Debt,TM,14699965.0,0.0,0.0,0.0,0.0,0,1,0,0,0


In [13]:
df2 = df2.reset_index()

In [14]:
# https://datatofish.com/if-condition-in-pandas-dataframe/

In [15]:
df2.head()

,OrderNbr,BranchID,Position,SlsperID,DateOfOrder,SlsName,Terms,PaymentsForm,OrdAmtRelease2,DeliveredOrderAmt,ReturnOrdAmt,DebConfirmAmt,DebConfirmAmtRelease,CountReturnOrd,CountOrdRelease2,DeliveredOrder,CountDebtConfirm,CountDebtConfirmRelease
0,DH012017-13345,MR0001,LOG,MR0456,2017-01-18,Lê Hồng Phúc,Debt,TM,19749975.0,0.0,0.0,0.0,0.0,0,1,0,0,0
1,DH012017-13346,MR0001,LOG,MR0456,2017-01-18,Lê Hồng Phúc,Debt,TM,24359987.0,0.0,0.0,0.0,0.0,0,1,0,0,0
2,DH012017-13360,MR0001,LOG,MR0456,2017-01-18,Lê Hồng Phúc,Debt,TM,17499997.0,0.0,0.0,0.0,0.0,0,1,0,0,0
3,DH012019-05622,MR0003,LOG,MR2168,2019-01-10,Nguyễn Anh Tuấn,Debt,TM,6150000.0,0.0,0.0,0.0,0.0,0,1,0,0,0
4,DH012020-00503,MR0016,MDS,MR1732,2020-01-02,Lâm Ngô Cường,Debt,TM,14699965.0,0.0,0.0,0.0,0.0,0,1,0,0,0


In [16]:
rename_dict ={'OrdAmtRelease2':'tienchotso', 'DeliveredOrderAmt':'tiengiaothanhcong', 'ReturnOrdAmt':'tienhuydon',
'DebConfirmAmt':'tienlenbangke', 'DebConfirmAmtRelease':'tienthuquyxacnhan'
}
df2.rename(columns=rename_dict, inplace=True)

In [17]:
# df2['tienchotso'] = np.where(df2['PaymentsForm'].values=='TM', df2['OrdAmtRelease2'].values, 0)
# df2['tiengiaothanhcong'] = np.where(df2['PaymentsForm'].values=='TM', df2['DeliveredOrderAmt'].values, 0)
# df2['tienhuydon'] = np.where(df2['PaymentsForm'].values=='TM', df2['ReturnOrdAmt'].values, 0)
# df2['tienlenbangke'] = np.where(df2['PaymentsForm'].values=='TM', df2['DebConfirmAmt'].values, 0)
# df2['tienthuquyxacnhan'] = np.where(df2['PaymentsForm'].values=='TM', df2['DebConfirmAmtRelease'].values, 0)
df2['chitietnocongty'] = df2['tienchotso'].values - df2['tienhuydon'].values - df2['tienthuquyxacnhan'].values
df2['tiennocongty'] = np.where(df2['chitietnocongty']>0, df2['chitietnocongty'].values, 0)
# df2['tienchotso'] = df2.apply(lambda x: x['OrdAmtRelease2'] if x['PaymentsForm']=='TM' else 0, axis=1)
# df2['tiengiaothanhcong'] = df2.apply(lambda x: x['DeliveredOrderAmt'] if x['PaymentsForm']=='TM' else 0, axis=1)
# df2['tienhuydon'] = df2.apply(lambda x: x['ReturnOrdAmt'] if x['PaymentsForm']=='TM' else 0, axis=1)
# df2['tienlenbangke'] = df2.apply(lambda x: x['DebConfirmAmt'] if x['PaymentsForm']=='TM' else 0, axis=1)
# df2['tienthuquyxacnhan'] = df2.apply(lambda x: x['DebConfirmAmtRelease'] if x['PaymentsForm']=='TM' else 0, axis=1)
# df2['chitietnocongty'] = df2['tienchotso'] - df2['tienhuydon'] - df2['tienthuquyxacnhan']
# df2['tiennocongty'] = df2.apply(lambda x: x['chitietnocongty'] if x['chitietnocongty']>0 else 0, axis=1)

In [18]:
rename_dict ={'CountOrdRelease2':'donchotso', 'DeliveredOrder':'dongiaothanhcong', 'CountReturnOrd':'donhuy',
'CountDebtConfirm':'donlenbangke', 'CountDebtConfirmRelease':'donthuquyxacnhan'
}

In [19]:
df2.rename(columns=rename_dict, inplace=True)

In [20]:
# df2['donchotso'] = df2.apply(lambda x: x['CountOrdRelease2'], axis=1)
# df2['dongiaothanhcong'] = df2.apply(lambda x: x['DeliveredOrder'] if x['PaymentsForm']=='TM' else 0, axis=1)
# df2['donlenbangke'] = df2.apply(lambda x: x['CountDebtConfirm'] if x['PaymentsForm']=='TM' else 0, axis=1)
# df2['donthuquyxacnhan'] = df2.apply(lambda x: x['CountDebtConfirmRelease'] if x['PaymentsForm']=='TM' else 0, axis=1)
df2['chitietdonnocongty'] = df2['donchotso'] - df2['donthuquyxacnhan'] - df2['donhuy']
df2['donnocongty_old'] = df2.apply(lambda x: x['chitietdonnocongty'] if x['chitietdonnocongty']>0 else 0, axis=1)
df2['donnocongty'] = np.where(df2['tiennocongty'].values > 0, 1, 0)

In [21]:
df2.columns

Index(['OrderNbr', 'BranchID', 'Position', 'SlsperID', 'DateOfOrder',
       'SlsName', 'Terms', 'PaymentsForm', 'tienchotso', 'tiengiaothanhcong',
       'tienhuydon', 'tienlenbangke', 'tienthuquyxacnhan', 'donhuy',
       'donchotso', 'dongiaothanhcong', 'donlenbangke', 'donthuquyxacnhan',
       'chitietnocongty', 'tiennocongty', 'chitietdonnocongty',
       'donnocongty_old', 'donnocongty'],
      dtype='object')

In [22]:
col_list = ['OrderN', 'BranchID', 'Position', 'SlsperID', 'Ngày Tạo Đơn', 'MDS', 'Terms',
       'PaymentsForm', 'Tiền Chốt Sổ',
       'Tiền Giao Thành Công', 'Tiền Hủy Đơn', 'Tiền Lên Bảng Kê',
       'Tiền Thủ Quỹ Xác Nhận', 'Don Huy', 'Đơn Chốt Sổ', 'Đơn Giao Thành Công',
       'Đơn Lên Bảng Kê', 'Đơn Thủ Quỹ Xác Nhận', 
       'Chi Tiết Tiền Nợ Công Ty', 'Tiền Nợ Công Ty',
       'Chi Tiết Đơn Nợ Công Ty', 'Đơn Nợ Công Ty Old', 'Đơn Nợ Công Ty']

In [23]:
df2.columns = col_list
df2.to_csv('doanhthutienmat_data.csv', index=False)

In [24]:
# Google Sheet connection
scopes = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive',
          'https://www.googleapis.com/auth/drive.file']
jsonfile = 'D:/data_sale/datateam1599968716114-6f9f144b4262.json'
credentials = service_account.Credentials.from_service_account_file(jsonfile, scopes=scopes)
service = discovery.build('drive', 'v3', credentials=credentials)
#spreadsheets_id = '1qEwviiJcAtvWCLvc-5AXOgidaeHMsi2RWWIKiUehabI'
folder_id = '12iN7KEKpgT4QhIfaLNB5shQ4g5x-VUoY'
file_name = 'doanhthutienmat_data.csv'
file_type = 'text/csv'
file_body={'name':file_name,'addParents':[folder_id]}
query = f"parents = '{folder_id}'"
files = service.files().list(q=query).execute()
fileid  = files['files'][0]['id']

media = MediaFileUpload('{0}'.format(file_name), mimetype=file_type)
updated_file = service.files().update(fileId=fileid, body=file_body, media_body=media)
updated_file.execute()